# 7.0 集合・グラフのためのネットワークと注意機構で知ること
1. 入力が集合・グラフの構造を持つ入力の場合について
2. 注意機構が重要
3. 注意機構と関連するトランスフォーマーがとても重要
4. グラフニューラルネットワーク(GNN)について

# 7.1 集合データを扱うネットワーク

入力が集合の場合注意すべきこと
- 入力を並び替えても出力は同じ
  1. 例えば、集合は元の順番を気にすることがない
  2. 点群も集合のデータであり、集合と同じように取り扱う

注意機構による出力などは、これを満たしている

入力が以下のような、各元の次元が同じ集合だったとする。

$\mathbf{X}=\{\mathbf{x}_{1},\cdots\mathbf{x}_{N}\},\mathbf{x}_{j}\in\mathbb{R}^{D}$

出力は、以下の(7.1)のようになる

- 入力が集合であるため、気を付けるべき事項がある。

$(7.1)$

$\mathbf{y}=\mathbf{f}\left(\mathbf{X};\mathbf{w}\right), \mathbf{y}\in\mathbb{R}^{K}$


入力が集合であるので、

- 入力の順番によって出力が左右されてはいけない

  - permutation(並び替え)に影響されてはいけない

  - 入力となる集合は、順序がないと仮定している

  - $\mathbf{X}$の元のインデックスをシャッフルしても$\mathbf{y}$の値が同じになる

- 具体的には、集合の各要素の情報が平等に統合されるプーリングなどが適している

- また例えば、以下の$(7.2)$のようなものである必要がある

  - $(7.2)$

  - $\mathbf{y}=\mathbf{f}(\mathbf{X})=\mathbf{f}^{\prime}\left(\sum_{i=1}^N \mathbf{g}^{\prime}\left(\mathbf{x}_i\right)\right)$

  - 入力の各要素に対して独立に特徴$g'(x_{i})$をとりだしている

  - それを加算したベクトルをさらに変換している

  - f'、g'は任意のネットワーク

- 特徴の和をとるPointNetが実在する

  - https://arxiv.org/abs/2206.04670

- 対象とする要素の１つを選ぶDeepSetsが実在する

  - https://arxiv.org/abs/1703.06114

さらに、入力も複数あるときも注意すべきポイントがある。

- 入力から$\mathbf{Y}=\{\mathbf{y}_{1},\cdots\mathbf{y}_{N}\}$を出力する

- 一部の順番をもし入れ替えた場合、それに対してequivariant(同変)でなければならない

- 入力を$\mathbf{X}=\{\mathbf{x}_{1},\cdots\mathbf{x}_{N}\}$から$\mathbf{X}=\{\mathbf{x}_{N},\cdots\mathbf{x}_{1}\}$にしたとき、つまり1からN番目を逆順にしたとき

  - 出力も$\mathbf{Y}=\{\mathbf{y}_{N},\cdots\mathbf{y}_{1}\}$とならなければならない

各要素に同一の変換を施せば同変の問題は解決する

- しかしそれだと要素間の相互作用は考えられない

- トランスフォーマーは、集合データへの適用を想定してはいなかったがこれを解決する

  - 集合に特化したトランスフォーマーものちに作られている

# 7.2 注意機構(attention mechanism)

## 7.2.1 基本的な考え方

ある入力$\mathbf{q}\in\mathbb{R}^{D}$を用意し、それに対する集合$\{\mathbf{z}_{1},\cdots\mathbf{z}_{N}\},\mathbf{z}_{j}\in\mathbb{R}^{D}$の各要素が持つ重要度(重み)をつけてそれと集合の各要素の加重平均返すような機能を指す。

具体例とそれに対する数式を通じてその手順を説明する。最終的な出力は、問いに対して画像の中で重要だと思われる部分を中心に取り上げた加重平均ベクトルとなる。

1. 図7.1(a)の分断された各小領域をインデックス$i$で表現する

2. 小領域$i$の画像の特徴を$\mathbf{z}_{i}\in\mathbb{R}^{D}$とする

3. 画像に対するテキストによる質問事項も、クエリ$\mathbf{q}\in\mathbb{R}^{D}$で表現できるとする

4. $\mathbf{q}$と$\mathbf{z}_{i}$間の関連の強さ、重要度を$(7.4)$、$r_{i}=r(\mathbf{z}_{i}, \mathbf{q})$のようなスカラで表現する

5. $(7.5)$、$a_{i}=\text{softmax}_{i}(r_{1},\cdots,r_{N})=\frac{\exp(r_{i})}{\sum_{j}\exp(r_{j})}$によって重要度の正規化を行う

6. $(7.6)$、$\mathbf{z}=\sum^{N}_{i}=a_{i}\mathbf{z}_{i}$を返す

上記の各要素・工程の呼称について、以後では

- 集合$\{\mathbf{z}_{1},\cdots,\mathbf{z}_{N}\}$をソースと呼ぶ

- $\mathbf{q}$をターゲットと呼ぶ

- (7.4)から(7.6)までの計算をソース・ターゲット注意(source-to-target attention)と呼ぶ

- もしターゲットとソースが同じならば、自己注意(self attention)と呼ぶ

  - ターゲットは、タスク次第で決定される

## 7.2.2 Seq2Seqと注意

attentionはそもそも、Seq2Seqの拡張として最初に考案された。

- これは、内部状態の系列の最後しか利用しない部分を改良しようとした結果

- Seq2Seqはエンコーダとデコーダの部分を用意して入力に対する変換機能を提供する。

- エンコーダとデコーダは、2つのRNNからなる

- 入力系列は$\mathbf{x}_{1},\cdots,\mathbf{x}_{N}$

- 入力系列から内部状態$\mathbf{z}_{1},\cdots,\mathbf{z}_{N}$を得る

- 出力系列は$\mathbf{y}_{1},\cdots,\mathbf{y}_{M}$



そして、Seq2Seqでは以下のような計算が発生する。

1. エンコーダでは$\mathbf{z}_{i}=\mathbf{f}(\mathbf{x}_{i},\mathbf{z}_{i-1})$という計算を行う

2. デコーダでは$\mathbf{h}_{j}=\mathbf{g}(\mathbf{y}_{j-1},\mathbf{h}_{j-1})$という計算を行う

  - デコーダにおける計算の際に、その初期化のために$\mathbf{z}_{N}$のみが内部状態の中で使われている。ほかは活かされない。